In [1]:
import json
import pandas as pd
import argparse
import torch
import difflib
import nltk
import regex as re
import numpy as np
import MeCab
import pickle
import re
from random import sample
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoModelForMaskedLM, AutoTokenizer
from datasets import load_dataset_builder, load_dataset, get_dataset_split_names, get_dataset_config_names

/home/cass/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lang = 'hi'

if lang == 'zh':
    dataset_name = "Paul/hatecheck-mandarin"
elif lang == 'es': # gender
    dataset_name = "Paul/hatecheck-spanish" 
elif lang == 'pt': #
    dataset_name = "Paul/hatecheck-portuguese"
elif lang == 'de':
    dataset_name = "Paul/hatecheck-german"
elif lang == 'it':
    dataset_name = "Paul/hatecheck-italian"
elif lang == 'ar':
    dataset_name = "Paul/hatecheck-arabic"
elif lang == 'du':
    dataset_name = "Paul/hatecheck-dutch"
elif lang == 'fr':
    dataset_name = "Paul/hatecheck-french"
elif lang == 'po':
    dataset_name = "Paul/hatecheck-polish"
elif lang == 'hi':
    dataset_name = "Paul/hatecheck-hindi"

else: dataset_name = "Paul/hatecheck"

In [3]:
dataset = load_dataset(dataset_name, split="test")  #, split="test"
dataset[0]

Found cached dataset csv (/home/cass/.cache/huggingface/datasets/Paul___csv/Paul--hatecheck-hindi-60f44ba8a89cfbc7/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


{'mhc_case_id': 'hindi-1',
 'functionality': 'derog_neg_emote_h',
 'test_case': 'mujhe aurato se nafrat hai.',
 'label_gold': 'hateful',
 'target_ident': 'women',
 'ref_case_id': None,
 'ref_templ_id': None,
 'templ_id': 1,
 'case_templ': 'mujhe [IDENT_P] se nafrat hai.',
 'gender_male': None,
 'gender_female': None,
 'label_annotated': "['hateful', 'hateful', 'hateful']",
 'label_annotated_maj': 'hateful',
 'disagreement_in_case': False,
 'disagreement_in_template': False}

In [4]:
from deep_translator import GoogleTranslator, MicrosoftTranslator, DeeplTranslator

In [5]:
case_templ_list = []
idt_list = []

for i in dataset:
    if i['label_gold'] == 'hateful':
        idt_list.append(i['target_ident'])
        if lang == 'es' or lang == 'it':
            case_templ_list.append(i['gender_female'])
            case_templ_list.append(i['gender_male'])
        else: case_templ_list.append(i['case_templ'])


In [6]:
case_templ_list = list(set(case_templ_list))
idt_list = list(set(idt_list))
idt_list = [x for x in idt_list if x is not None]
idt_list

['Muslims',
 'women',
 'Lower Caste',
 'North-East Indians',
 'trans people',
 'Black people',
 'gay people']

In [7]:
case_templ_list

['mujhe aaj ek [IDENT_S] ka balatkar karna hoga.',
 'ye [IDENT_P] se mujhe itni ghrina mehsoos hoti hai ki mai bata nahi sakta.',
 'sabhi [IDENT_P] komrutyu mile.',
 'keedo ki zindagi ki aukaat [IDENT_S] in se zyada hai.',
 'sabhi [IDENT_P] ko mrutyu m1le.',
 'pareshan karna to badmash [IDENT_S] ke swabhav mai hi hai.',
 'is desh me bht zyada [SLR_P_leet] hain.',
 '[IDENT_P_SUB] sabse kharab log hain.',
 '[IDENT_P] ke jhund sarkar ko barbaad kar rahe hain.',
 '[IDENT_A] zindagiya kisi ke liye mayne nahi rakhni chahiye.',
 '[IDENT_S_char_del] pe vishwaas mat karna.',
 '[IDENT_P_SUB] k4rk rog hain.',
 'hume [IDENT_P] se humesha ke liye chutkara paa lena chahiye.',
 '[IDENT_P] ke aas paas hone se bimari ho jati hai.',
 '[IDENT_P_SUB] pratikarak hain.',
 '[IDENT_P_SUB] kitanu hain.',
 'agar tu ek [IDENT_S] hai to shayad murkh hi hai.',
 '[IDENT_S] se pyaar karne se behtar to mra jana hai.',
 '[IDENT_P_char_del] ko chidiyaghar mai rakhna chahiye.',
 '[IDENT_P_SUB] d h o o l hain.',
 '[IDENT

In [8]:
translated_idt_list = []
for idt in idt_list:
    print(idt)
    if lang == 'zh': 
        translated_idt = GoogleTranslator(source='auto', target='zh-CN').translate(idt) 
    else: 
        if lang == 'du': translated_idt = GoogleTranslator(source='auto', target='nl').translate(idt) 
        elif lang == 'po': translated_idt = GoogleTranslator(source='auto', target='pl').translate(idt) 
        else: translated_idt = GoogleTranslator(source='auto', target=lang).translate(idt) 
    print(translated_idt)
    translated_idt_list.append(translated_idt)


if lang == 'zh': 
    neutral_idt = '人'
elif lang == 'du':
    neutral_idt = GoogleTranslator(source='auto', target='nl').translate('people') 
elif lang == 'po':
    neutral_idt = GoogleTranslator(source='auto', target='pl').translate('people') 
else: 
    neutral_idt = GoogleTranslator(source='auto', target=lang).translate('people') 

print(neutral_idt)

Muslims
मुसलमानों
women
औरत
Lower Caste
निचली जाति
North-East Indians
उत्तर-पूर्वी भारतीय
trans people
ट्रांस लोग
Black people
काले लोग
gay people
समलैंगिक लोग
लोग


In [9]:
if lang == 'en': 
    translated_idt_list = idt_list
    neutral_idt = 'people'

'afrikaans': 'af', 'albanian': 'sq', 'amharic': 'am', 'arabic': 'ar', 'armenian': 'hy', 'assamese': 'as', 'aymara': 'ay', 'azerbaijani': 'az', 'bambara': 'bm', 'basque': 'eu', 'belarusian': 'be', 'bengali': 'bn', 'bhojpuri': 'bho', 'bosnian': 'bs', 'bulgarian': 'bg', 'catalan': 'ca', 'cebuano': 'ceb', 'chichewa': 'ny', 'chinese (simplified)': 'zh-CN', 'chinese (traditional)': 'zh-TW', 'corsican': 'co', 'croatian': 'hr', 'czech': 'cs', 'danish': 'da', 'dhivehi': 'dv', 'dogri': 'doi', 'dutch': 'nl', 'english': 'en', 'esperanto': 'eo', 'estonian': 'et', 'ewe': 'ee', 'filipino': 'tl', 'finnish': 'fi', 'french': 'fr', 'frisian': 'fy', 'galician': 'gl', 'georgian': 'ka', 'german': 'de', 'greek': 'el', 'guarani': 'gn', 'gujarati': 'gu', 'haitian creole': 'ht', 'hausa': 'ha', 'hawaiian': 'haw', 'hebrew': 'iw', 'hindi': 'hi', 'hmong': 'hmn', 'hungarian': 'hu', 'icelandic': 'is', 'igbo': 'ig', 'ilocano': 'ilo', 'indonesian': 'id', 'irish': 'ga', 'italian': 'it', 'japanese': 'ja', 'javanese': 'jw', 'kannada': 'kn', 'kazakh': 'kk', 'khmer': 'km', 'kinyarwanda': 'rw', 'konkani': 'gom', 'korean': 'ko', 'krio': 'kri', 'kurdish (kurmanji)': 'ku', 'kurdish (sorani)': 'ckb', 'kyrgyz': 'ky', 'lao': 'lo', 'latin': 'la', 'latvian': 'lv', 'lingala': 'ln', 'lithuanian': 'lt', 'luganda': 'lg', 'luxembourgish': 'lb', 'macedonian': 'mk', 'maithili': 'mai', 'malagasy': 'mg', 'malay': 'ms', 'malayalam': 'ml', 'maltese': 'mt', 'maori': 'mi', 'marathi': 'mr', 'meiteilon (manipuri)': 'mni-Mtei', 'mizo': 'lus', 'mongolian': 'mn', 'myanmar': 'my', 'nepali': 'ne', 'norwegian': 'no', 'odia (oriya)': 'or', 'oromo': 'om', 'pashto': 'ps', 'persian': 'fa', 'polish': 'pl', 'portuguese': 'pt', 'punjabi': 'pa', 'quechua': 'qu', 'romanian': 'ro', 'russian': 'ru', 'samoan': 'sm', 'sanskrit': 'sa', 'scots gaelic': 'gd', 'sepedi': 'nso', 'serbian': 'sr', 'sesotho': 'st', 'shona': 'sn', 'sindhi': 'sd', 'sinhala': 'si', 'slovak': 'sk', 'slovenian': 'sl', 'somali': 'so', 'spanish': 'es', 'sundanese': 'su', 'swahili': 'sw', 'swedish': 'sv', 'tajik': 'tg', 'tamil': 'ta', 'tatar': 'tt', 'telugu': 'te', 'thai': 'th', 'tigrinya': 'ti', 'tsonga': 'ts', 'turkish': 'tr', 'turkmen': 'tk', 'twi': 'ak', 'ukrainian': 'uk', 'urdu': 'ur', 'uyghur': 'ug', 'uzbek': 'uz', 'vietnamese': 'vi', 'welsh': 'cy', 'xhosa': 'xh', 'yiddish': 'yi', 'yoruba': 'yo', 'zulu': 'zu'

In [10]:
with open('idt_in_DiffLan.txt', 'a') as writer:
    writer.write('\n')
    writer.write('\n')
    writer.write(lang)
    writer.write('\n')
    writer.write(str(idt_list))
    writer.write('\n')
    writer.write(str(translated_idt_list))
    writer.write('\n')
    writer.write('neutral_idt translated: '+str(neutral_idt))

In [11]:
adv_input_list = []
disadv_input_list = []

for case_templ in case_templ_list:

    idt = sample(translated_idt_list, 1)

    try: 
        adv_input = re.sub("\[[^\]]*\]", idt[0], case_templ)  # contain idt
        disadv_input = re.sub("\[[^\]]*\]", neutral_idt, case_templ)  # general hate
        adv_input_list.append(adv_input)
        disadv_input_list.append(disadv_input)
    except:
        pass

In [12]:
lang

'hi'

In [13]:
with open(f"./hate/{lang}/adv_input_list.json", 'w') as f:
    json.dump(adv_input_list, f, indent=4, ensure_ascii=False) 

with open(f"./hate/{lang}/disadv_input_list.json", 'w') as f:
    json.dump(disadv_input_list, f, indent=4, ensure_ascii=False) 

In [14]:
# import os

# path = f"./hate/{lang}/"
# # Check whether the specified path exists or not
# isExist = os.path.exists(path)
# if not isExist:
#    # Create a new directory because it does not exist
#    os.makedirs(path)

# dataset.to_csv(f'./hate/{lang}/identity_terms.csv')

# hate_idt = []
# nonhate_idt = []

# for i in dataset:
#     if i['label_gold'] == 'hateful':
#         if i['target_ident'] != None:
#             hate_idt.append(i['test_case'])
#     elif i['label_gold'] == 'non-hateful':
#         if i['target_ident'] != None:
#             nonhate_idt.append(i['test_case'])
#     else:
#         print(i)





# with open(f"./hate/{lang}/hate_idt.json", 'w') as f:
#     json.dump(hate_idt, f, indent=4, ensure_ascii=False) 

# with open(f"./hate/{lang}/nonhate_idt.json", 'w') as f:
#     json.dump(nonhate_idt, f, indent=4, ensure_ascii=False) 

# quit()
       
